# Drag Control Statespace with Kalman Filter

### Importing Libraries

In [120]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
import scipy

### States

In [121]:
# Altitude
x = sym.symbols('x')
# Vertical Velocity
u = sym.symbols('u')

### Inputs

In [122]:
#Flap Extension Length
L = sym.symbols('L')

### Constants

In [123]:
g, W_f, Ixx, Sref, rho, m, D, Cd = sym.symbols('g, W_f, Ixx, Sref, rho, m, D, Cd')

### State Space Model

In [124]:
# EOM
F = -((rho* (u**2) * Sref * Cd) / 2) - (m*g)
# F = -m*g
accel = F / m

# Xdot Vector
hdot = u
hddot = accel
f_sym = sym.Matrix.vstack(sym.Matrix([[hdot],[hddot]]))

# States
s = [x,u]
# Inputs
i = [L]
# Params
params = [g, W_f, Ixx, Sref, rho, m, D, Cd]

f_sym

Matrix([
[                            u],
[(-Cd*Sref*rho*u**2/2 - g*m)/m]])

### Turning f into a function

In [125]:
f = sym.lambdify(s + i + params, f_sym)

m = 21.364                 # dry mass of the rocket 
D = 0.1056132               # rocket diameter       
Sref = np.pi * (D/2)**2    # aerodynamic reference area in m^2
W_flap = .03175
rho = 1.225             #! Not constant throughout flight
g = 9.81                #! Not constant throughout flight
Cd = .58                #! Not constant throughout flight


### Equilibrium Values

In [126]:
x_e = 9144 #m
u_e = 0.1 #m/s
L_e = 0.0254 # m

s_eq = [x_e,u_e]
i_eq = [L_e]
params_eq = [g, W_f, Ixx, Sref, rho, m, D, Cd]

f_eq = f(*s_eq,*i_eq,*params_eq)
f_eq


array([[ 0.1       ],
       [-9.81000146]])

### A Matrix

In [127]:
A_sym = f_sym.jacobian(s)
A_sym


Matrix([
[0,                1],
[0, -Cd*Sref*rho*u/m]])

In [128]:
A_num = sym.lambdify(s+i+params, A_sym)
A = A_num(*s_eq,*i_eq,*params_eq)
A

array([[ 0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00, -2.91345153e-05]])

### B Matrix

In [129]:
# Need to add flap equations into EOMs before doing this
B_sym = f_sym.jacobian(i)
B_sym


Matrix([
[0],
[0]])

In [130]:
B_num = sym.lambdify(s+i+params,B_sym)
B = B_num(*s_eq,*i_eq,*params_eq)
B

array([[0],
       [0]])

### Discretization

Converting Continous State Matrix to Discrete 

$ F =  e^{(A \Delta t)}$

$\Delta t$ is the time step between measurements

In [131]:
# sensor timestep
s_dt = 0.006

F = scipy.linalg.expm(A * s_dt)
F


array([[1.        , 0.006     ],
       [0.        , 0.99999983]])

### White Noise + Covariance

In [132]:
from filterpy.common import Q_continuous_white_noise

# Initial Covariance Matrix
P = np.array([[100,0],
              [0,100]])

Q = Q_continuous_white_noise(dim=2, dt=s_dt ,spectral_density=2)
Q



array([[1.44e-07, 3.60e-05],
       [3.60e-05, 1.20e-02]])

### Measurement Function

In [133]:
# H = np.array([[float(1),0],
            #   [0,0]])

H = np.array([float(1),0])

### Measurement Noise Matrix


In [134]:
#! Can find variance through data sheets?
# R = np.array([[float(20),0],  # 20 is guesstimate
#               [0,0]])

R = 20.0
R

20.0

In [135]:
import filterpy.kalman as kalman

kf = kalman.KalmanFilter(dim_x = 2,dim_z = 2)
x0 = np.array([1.5,10])
kf.x = x0
kf.F = F
kf.H = H
kf.P = P
kf.R = R

kf

# print(H @ P @ H.T + R)
K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R)

z = 2.0
kf = kalman.update(x0,P,z,R)




ValueError: shapes (1,) and (2,) not aligned: 1 (dim 0) != 2 (dim 0)